In [1]:
%env TOKENIZERS_PARALLELISM=false
!pip install bertopic

env: TOKENIZERS_PARALLELISM=false
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 53.2 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=819467 sha256=e025e37848b964ab54043c9c62daf3871ddff6ee8bf06536aebb2153b28517d4
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=a60720fa5318383fe724cf16fceaceb422a9b506f40c443b1eddd1c7506cf4e2
  Stored in directory: /root/.cache/pip/wheels/62/f2

In [2]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/stranger-things-dialogue-dataset/stranger_things_all_dialogue.csv')
df['text'] = df['raw_text'] + ' ' + df['stage_direction']
df.head()

,season,episode,line,raw_text,stage_direction,dialogue,start_time,end_time,text
0,1,1,1,[crickets chirping],[crickets chirping],NaN,00:00:07,00:00:09,[crickets chirping] [crickets chirping]
1,1,1,2,[alarm blaring],[alarm blaring],NaN,00:00:49,00:00:51,[alarm blaring] [alarm blaring]
2,1,1,3,[panting],[panting],NaN,00:00:52,00:00:54,[panting] [panting]
3,1,1,4,[elevator descending],[elevator descending],NaN,00:01:01,00:01:02,[elevator descending] [elevator descending]
4,1,1,5,[elevator dings],[elevator dings],NaN,00:01:09,00:01:10,[elevator dings] [elevator dings]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32519 entries, 0 to 32518
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   season           32519 non-null  int64 
 1   episode          32519 non-null  int64 
 2   line             32519 non-null  int64 
 3   raw_text         32519 non-null  object
 4   stage_direction  10678 non-null  object
 5   dialogue         26435 non-null  object
 6   start_time       32519 non-null  object
 7   end_time         32519 non-null  object
 8   text             10678 non-null  object
dtypes: int64(3), object(6)
memory usage: 2.2+ MB


In [4]:
from collections import Counter
text = ' '.join(df['text'].dropna().values).replace('[', '').replace(']', '')
counts = Counter(text.split())
print(counts.most_common(n=20))
unique = set(text.split())
print(len(unique))

[('music', 1801), ('playing', 1392), ('sighs', 764), ('in', 665), ('grunts', 637), ('chuckles', 548), ('continues', 478), ('panting', 477), ('Russian', 472), ('I', 460), ('Dustin', 454), ('Hopper', 416), ('Mike', 405), ('you', 390), ('Joyce', 365), ('door', 337), ('on', 330), ('Eleven', 313), ('Steve', 312), ('the', 307)]
6924


In [5]:
from collections import defaultdict
document_counts = defaultdict(int)
for index, row in df.dropna(subset=['text']).iterrows():
    current = set(row['text'].replace('[', '').replace(']', '').split())
    for token in current:
        document_counts[token] += 1
percentages = {key: value/len(df['text'].dropna()) for key, value in document_counts.items()}
list(percentages.items())[:10]

[('crickets', 0.00028095148904289194),
 ('chirping', 0.001311106948866829),
 ('alarm', 0.003652369357557595),
 ('blaring', 0.002341262408690766),
 ('panting', 0.023038022101517137),
 ('elevator', 0.001311106948866829),
 ('descending', 0.00018730099269526128),
 ('dings', 0.0014047574452144597),
 ('heavily', 0.009833302116501217),
 ('breathing', 0.013017418992320659)]

In [6]:
pcts = sorted(list(percentages.values()), reverse=True)
pcts[:10]

[0.08540925266903915,
 0.06518074545795093,
 0.038677654991571456,
 0.038209402509833304,
 0.03568083910844728,
 0.03493163513766623,
 0.03137291627645627,
 0.0278141974152463,
 0.025941187488293688,
 0.02519198351751264]